In [1]:
from autogluon.tabular import TabularPredictor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler
import pandas as pd
import joblib

In [2]:
ds = pd.read_csv("/workspace/predictionDataset.csv")
ds.head()

,Estimated_fire_area,Mean_estimated_fire_brightness,Mean_estimated_fire_radiative_power,Precipitation_PCA,RelativeHumidity_PCA,SoilWaterContent_PCA,SolarRadiation_PCA,Temperature_PCA,WindSpeed_PCA
0,8.68000,312.266667,42.400000,0.618025,0.824141,3.648440,3.109959,0.427663,1.119870
1,16.61125,322.475000,62.362500,0.572057,0.026436,3.188706,2.652126,0.663295,0.720070
2,5.52000,325.266667,38.400000,0.155604,0.627808,3.084696,2.542103,0.715487,1.429688
3,6.26400,313.870000,33.800000,0.669643,1.603821,2.697019,2.625345,1.033359,1.067146
4,5.40000,337.383333,122.533333,0.646868,1.351653,2.084853,2.656208,0.413631,0.935829


In [3]:
ds.describe()

,Estimated_fire_area,Mean_estimated_fire_brightness,Mean_estimated_fire_radiative_power,Precipitation_PCA,RelativeHumidity_PCA,SoilWaterContent_PCA,SolarRadiation_PCA,Temperature_PCA,WindSpeed_PCA
count,26660.000000,26660.000000,26660.000000,26660.000000,26660.000000,26660.000000,26660.000000,26660.000000,26660.000000
mean,143.864447,319.708774,83.709296,0.919174,1.699032,1.528229,1.783703,1.773529,1.704073
std,313.529799,8.889730,67.869521,1.629660,0.904842,0.989675,0.891817,0.911741,0.973766
min,1.000000,290.700000,0.000000,0.002240,0.021841,0.008024,0.035191,0.033183,0.047228
25%,8.819688,313.956371,44.119545,0.454932,1.034660,0.739419,1.147684,1.045389,1.062091
50%,38.218684,319.843849,67.130556,0.664544,1.568046,1.348292,1.653960,1.675570,1.495117
75%,146.434093,325.469297,103.158324,0.743413,2.194136,2.089933,2.263872,2.415666,2.086959
max,10120.943170,381.950000,2178.600000,44.070996,5.912656,6.214919,11.085709,6.058840,11.554963


In [4]:
# Create train, validation and test sets
train, test = train_test_split(ds, test_size=0.2, random_state=42)
y_train = train["Estimated_fire_area"]
y_test = test["Estimated_fire_area"]
X_train = train.drop("Estimated_fire_area", axis=1)
X_test = test.drop("Estimated_fire_area", axis=1)

# Scale data
train_scaler = RobustScaler()
test_scaler = RobustScaler()

train = train_scaler.fit_transform(X_train)
test = train_scaler.transform(X_test)
y_train = test_scaler.fit_transform(y_train.values.reshape(-1, 1))
y_test = test_scaler.transform(y_test.values.reshape(-1, 1))

train = pd.DataFrame(train, columns=X_train.columns)
test = pd.DataFrame(test, columns=X_test.columns)
y_train = pd.DataFrame(y_train, columns=["Estimated_fire_area"])
y_test = pd.DataFrame(y_test, columns=["Estimated_fire_area"])

joblib.dump(train_scaler, "/workspace/train_scaler.pkl")
joblib.dump(test_scaler, "/workspace/test_scaler.pkl")

train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)
y_train.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

train["Estimated_fire_area"] = y_train
test["Estimated_fire_area"] = y_test

In [5]:
train.head()

,Mean_estimated_fire_brightness,Mean_estimated_fire_radiative_power,Precipitation_PCA,RelativeHumidity_PCA,SoilWaterContent_PCA,SolarRadiation_PCA,Temperature_PCA,WindSpeed_PCA,Estimated_fire_area
0,0.446800,0.251043,0.275774,0.322184,0.807082,0.660935,0.089029,-0.467542,3.964015
1,-0.749654,-0.420758,-1.310796,1.353922,1.295682,0.878194,1.027563,0.243304,-0.044648
2,-0.371779,-0.173817,6.650278,-0.399380,-0.312342,-0.051805,-0.830166,-0.897210,-0.026718
3,0.582529,-0.089962,0.326392,1.681929,0.759647,0.568807,-0.768943,1.894185,2.421589
4,-0.847256,-0.698615,0.197132,2.249799,0.337001,1.496432,1.683282,0.369840,-0.268072


In [6]:
train.describe()

,Mean_estimated_fire_brightness,Mean_estimated_fire_radiative_power,Precipitation_PCA,RelativeHumidity_PCA,SoilWaterContent_PCA,SolarRadiation_PCA,Temperature_PCA,WindSpeed_PCA,Estimated_fire_area
count,2.132800e+04,2.132800e+04,2.132800e+04,2.132800e+04,21328.000000,2.132800e+04,21328.000000,21328.000000,21328.000000
mean,-1.358304e-02,2.780814e-01,8.859519e-01,1.163166e-01,0.132519,1.157132e-01,0.071440,0.202367,0.765990
std,7.764497e-01,1.125394e+00,5.651811e+00,7.815431e-01,0.730002,7.976408e-01,0.663482,0.944063,2.300250
min,-2.538110e+00,-1.142631e+00,-2.309346e+00,-1.326830e+00,-0.988776,-1.448847e+00,-1.195792,-1.407765,-0.268072
25%,-5.147597e-01,-3.918708e-01,-7.238621e-01,-4.575369e-01,-0.451162,-4.537689e-01,-0.458867,-0.421017,-0.212362
50%,-2.470795e-15,-1.204142e-16,1.936296e-16,-9.546400e-17,0.000000,9.936035e-17,0.000000,0.000000,0.000000
75%,4.852403e-01,6.081292e-01,2.761379e-01,5.424631e-01,0.548838,5.462311e-01,0.541133,0.578983,0.787638
max,5.394559e+00,2.080565e+01,1.514075e+02,3.738526e+00,3.496766,8.440963e+00,3.185314,9.798662,71.848099


In [7]:
predictor = TabularPredictor(
    label="Estimated_fire_area",
    problem_type="regression",
    eval_metric="root_mean_squared_error",
    path="/workspace/autogluonModels/iter6/",
).fit(
    time_limit=7200,
    presets="best_quality",
    train_data=train,
    ag_args_fit={"num_gpus": 1},
)

Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=8, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 7200s
AutoGluon will save models to "/workspace/autogluonModels/iter6/"
AutoGluon Version:  0.8.2
Python Version:     3.10.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Fri Jan 27 02:56:13 UTC 2023
Disk Space Avail:   269.30 GB / 499.32 GB (53.9%)
Train Data Rows:    21328
Train Data Columns: 8
Label Column: Estimated_fire_area
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    1367.3 MB
	Train Data (Original)  Memory Usage: 1.37 MB (0.1% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitt

In [8]:
predictor = TabularPredictor.load("/workspace/autogluonModels/iter6")

In [9]:
predictor.leaderboard(test)

                     model  score_test  score_val  pred_time_test  pred_time_val     fit_time  pred_time_test_marginal  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0      WeightedEnsemble_L3   -1.558427  -1.882489       32.835097      35.765360  4020.296632                 0.009124                0.001247           0.346581            3       True         17
1        LightGBMXT_BAG_L2   -1.559712  -1.883598       29.711218      34.778012  3691.151535                 3.836365                2.774222         355.871281            2       True         11
2      WeightedEnsemble_L2   -1.564321  -1.876932       22.358809      30.200645  1330.158630                 0.013092                0.001065           0.567433            2       True         10
3          LightGBM_BAG_L2   -1.564417  -1.895914       28.513145      32.767405  3663.172198                 2.638293                0.763615         327.891943            2       True         12
4          CatB

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,-1.558427,-1.882489,32.835097,35.765360,4020.296632,0.009124,0.001247,0.346581,3,True,17
1,LightGBMXT_BAG_L2,-1.559712,-1.883598,29.711218,34.778012,3691.151535,3.836365,2.774222,355.871281,2,True,11
2,WeightedEnsemble_L2,-1.564321,-1.876932,22.358809,30.200645,1330.158630,0.013092,0.001065,0.567433,2,True,10
3,LightGBM_BAG_L2,-1.564417,-1.895914,28.513145,32.767405,3663.172198,2.638293,0.763615,327.891943,2,True,12
4,CatBoost_BAG_L2,-1.566510,-1.877406,27.104779,32.292017,4134.718587,1.229927,0.288227,799.438333,2,True,14
5,LightGBMXT_BAG_L1,-1.568740,-1.883301,11.404705,19.985577,507.972153,11.404705,19.985577,507.972153,1,True,3
6,LightGBMLarge_BAG_L1,-1.576798,-1.904632,6.690495,6.375474,495.157728,6.690495,6.375474,495.157728,1,True,9
7,LightGBMLarge_BAG_L2,-1.577772,-1.915586,29.502133,33.365612,3817.767510,3.627281,1.361822,482.487256,2,True,16
8,LightGBM_BAG_L1,-1.581856,-1.902594,4.060882,3.045213,326.425317,4.060882,3.045213,326.425317,1,True,4
9,XGBoost_BAG_L1,-1.593130,-1.920274,5.326965,1.357976,472.576749,5.326965,1.357976,472.576749,1,True,8


In [10]:
x = predictor.evaluate(test)

Evaluation: root_mean_squared_error on test data: -1.5643206504517566
	Note: Scores are always higher_is_better. This metric score can be multiplied by -1 to get the metric value.
Evaluations on test data:
{
    "root_mean_squared_error": -1.5643206504517566,
    "mean_squared_error": -2.447099097429807,
    "mean_absolute_error": -0.7578311919955587,
    "r2": 0.3543088133476021,
    "pearsonr": 0.5976457008955776,
    "median_absolute_error": -0.3158463095841908
}


In [11]:
predictor.get_model_best()

'WeightedEnsemble_L2'

In [12]:
predictor.feature_importance(ds)

Computing feature importance via permutation shuffling for 8 features using 5000 rows with 5 shuffle sets...
	877.28s	= Expected runtime (175.46s per shuffle set)
	471.66s	= Actual runtime (Completed 5 of 5 shuffle sets)


,importance,stddev,p_value,n,p99_high,p99_low
Mean_estimated_fire_radiative_power,0.017217,0.001833,0.000015,5,0.020992,0.013443
Precipitation_PCA,0.003031,0.003031,0.044486,5,0.009272,-0.003209
RelativeHumidity_PCA,0.001171,0.005235,0.321616,5,0.011951,-0.009609
SolarRadiation_PCA,-0.000472,0.001891,0.696815,5,0.003422,-0.004366
Mean_estimated_fire_brightness,-0.001444,0.000476,0.998772,5,-0.000465,-0.002424
Temperature_PCA,-0.002061,0.001261,0.989167,5,0.000535,-0.004656
WindSpeed_PCA,-0.007270,0.002097,0.999254,5,-0.002952,-0.011588
SoilWaterContent_PCA,-0.015542,0.004079,0.999479,5,-0.007143,-0.023941


In [13]:
predictor.plot_ensemble_model()

'/workspace/autogluonModels/iter6/ensemble_model.png'